# Imports & Load Data 🌄

In [1]:
!pip install -q -U datasets[vision]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import numpy as np
import os
import glob
import shutil
import cv2
import matplotlib.pyplot as plt
import random
from typing import List, Tuple

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from google.colab import userdata

os.environ['WANDB_DISABLED'] = 'true'
roboflow_api = userdata.get('ROBOFLOW')

In [5]:
!pip install -q roboflow

from roboflow import Roboflow
rf = Roboflow(api_key=roboflow_api)
project = rf.workspace("potholedetection-86cjl").project("pothole_segmentation")
version = project.version(8)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.0 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...
[WARNING] we noticed you are downloading a `yolov8` datasets but you don't have `ultralytics` installed. Roboflow `.deploy` supports only models trained with `ultralytics==8.0.196`, to intall it `pip install ultralytics==8.0.196`.



Extracting Dataset Version Zip to pothole_segmentation-8 in yolov8:: 100%|██████████| 6160/6160 [00:01<00:00, 4204.07it/s]


### Prepare Huggingface Dataset Object

In [6]:
def load_mask_vertices(mask_path: str, img_height:int, img_width: int)->List[dict]:
    with open(mask_path, 'r') as file:
      lines = file.readlines()
    polygon_labels = []
    for line in lines:
      values = line.strip().split(' ')
      class_label = int(values[0])
      polygon_coords = [(int(float(values[i]) * img_width), int(float(values[i + 1]) * img_height)) for i in range(1, len(values), 2)]
      polygon_labels.append({'class_label': 1, 'polygon_coords': polygon_coords})

    return polygon_labels

def generate_mask_from_vertices(vertices: List[dict], img_height:int, img_width: int)->np.ndarray:
    binary_mask = np.zeros((img_height,img_width), dtype=np.uint8)
    for polygon in vertices :
      polygon_coords = np.array(polygon['polygon_coords'],np.int32).reshape((-1, 1, 2))
      cv2.fillPoly(binary_mask,[polygon_coords],color=(1))
    return binary_mask

def load_mask(mask_path: str, img_height:int, img_width: int)->np.ndarray:
    vertices = load_mask_vertices(mask_path, img_height, img_width)
    mask = generate_mask_from_vertices(vertices, img_height, img_width)
    return mask

def clrd(text: str, color: str = None):
    color_codes = {
        'ok': '\033[92m',        # Green
        'error': '\033[91m',     # Red
        'warning': '\033[93m',   # Yellow
        'success': '\033[92m',   # Green
        'status': '\033[95m',    # Magenta
        'special': '\033[94m',   # Blue
        'log': '\033[96m',       # Cyan
        'reset': '\033[0m'       # Reset to default color
    }
    color_code = color_codes.get(color, color_codes['reset'])
    return f"{color_code}{text}{color_codes['reset']}"

In [7]:
TRAIN_DIR = r"/content/pothole_segmentation-8/train"
TEST_DIR = r"/content/pothole_segmentation-8/test"
VAL_DIR = r"/content/pothole_segmentation-8/valid"

#Store masks in png format !!!
for dir in [TRAIN_DIR, TEST_DIR, VAL_DIR]:
  hits, misses = 0, 0
  os.makedirs(dir + "/masks", exist_ok=True)
  for path in os.listdir(os.path.join(dir, "labels")):
    try:
      mask = load_mask(os.path.join(dir, "labels", path), 640, 640)
      cv2.imwrite(os.path.join(dir, "masks", '.'.join(path.split('.')[:-1])+'.png'), mask)
      hits += 1
    except Exception as e:
      misses += 1

    if misses > 20:
      break
  print(clrd(f"Loaded {hits} masks", 'ok'), end='')
  if misses > 0:
    print(clrd(f'with {misses} errors', 'error'))
  print()

Loaded 2443 masks
Loaded 314 masks
Loaded 317 masks


In [8]:
from datasets import Dataset, DatasetDict, Image

train_images = [os.path.join(TRAIN_DIR, "images", path) for path in os.listdir(os.path.join(TRAIN_DIR, "images"))]
train_masks = [os.path.join(TRAIN_DIR, "masks", path) for path in os.listdir(os.path.join(TRAIN_DIR, "masks"))]

val_images = [os.path.join(VAL_DIR, "images", path) for path in os.listdir(os.path.join(VAL_DIR, "images"))]
val_masks = [os.path.join(VAL_DIR, "masks", path) for path in os.listdir(os.path.join(VAL_DIR, "masks"))]

test_images = [os.path.join(TEST_DIR, "images", path) for path in os.listdir(os.path.join(TEST_DIR, "images"))]
test_masks = [os.path.join(TEST_DIR, "masks", path) for path in os.listdir(os.path.join(TEST_DIR, "masks"))]

In [9]:
def create_dataset(image_paths, label_paths):
    dataset = Dataset.from_dict({"image": sorted(image_paths),
                                "label": sorted(label_paths)})
    dataset = dataset.cast_column("image", Image())
    dataset = dataset.cast_column("label", Image())

    return dataset

train_dataset = create_dataset(train_images, train_masks)
val_dataset = create_dataset(val_images, val_masks)
test_dataset = create_dataset(test_images, test_masks)

In [10]:
dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
})

In [11]:
import json
id2label = {1: 'pothole'}
with open('id2label.json', 'w') as fp:
    json.dump(id2label, fp)

In [12]:
label2id = {'pothole': 1}
with open('label2id.json', 'w') as fp:
    json.dump(label2id, fp)

# Train Seg-Transformers 🤗

In [13]:
!pip install -q -U transformers evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.1 MB/s eta 0:00:00


In [14]:
from transformers import AutoImageProcessor

checkpoint = "nvidia/mit-b0"
image_processor = AutoImageProcessor.from_pretrained(checkpoint, do_reduce_labels=True)

preprocessor_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

In [23]:
from torchvision.transforms import ColorJitter

jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1)

def train_transforms(example_batch):
    images = [jitter(x) for x in example_batch["image"]]
    labels = [x for x in example_batch["label"]]
    inputs = image_processor(images, labels)
    return inputs


def val_transforms(example_batch):
    images = [x for x in example_batch["image"]]
    labels = [x for x in example_batch["label"]]
    inputs = image_processor(images, labels)
    return inputs

In [24]:
train_ds = dataset["train"]
val_ds = dataset["validation"]

train_ds.set_transform(train_transforms)
val_ds.set_transform(val_transforms)

In [16]:
import evaluate
metric = evaluate.load("mean_iou")

In [17]:
import numpy as np
import torch
from torch import nn

def compute_metrics(eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred
        logits_tensor = torch.from_numpy(logits)
        logits_tensor = nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)

        pred_labels = logits_tensor.detach().cpu().numpy()
        metrics = metric.compute(
            predictions=pred_labels,
            references=labels,
            num_labels=1,
            ignore_index=255,
            reduce_labels=False,
        )
        for key, value in metrics.items():
            if isinstance(value, np.ndarray):
                metrics[key] = value.tolist()
        return metrics

In [18]:
from transformers import AutoModelForSemanticSegmentation, TrainingArguments, Trainer

model = AutoModelForSemanticSegmentation.from_pretrained(checkpoint, id2label=id2label, label2id=label2id)

config.json:   0%|          | 0.00/70.0k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/14.4M [00:00<?, ?B/s]

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="segformer-b0-scene-parse-150",
    learning_rate=6e-5,
    num_train_epochs=20,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    save_total_limit=3,
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    remove_unused_columns=False,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss,Validation Loss,Mean Iou,Mean Accuracy,Overall Accuracy,Per Category Iou,Per Category Accuracy
20,0.208200,0.068070,1.000000,1.000000,1.000000,[1.0],[1.0]
40,0.013000,0.031754,1.000000,1.000000,1.000000,[1.0],[1.0]
60,0.029000,0.021910,1.000000,1.000000,1.000000,[1.0],[1.0]
80,0.042000,0.012730,1.000000,1.000000,1.000000,[1.0],[1.0]
100,0.002200,0.016982,1.000000,1.000000,1.000000,[1.0],[1.0]
120,0.003900,0.022752,1.000000,1.000000,1.000000,[1.0],[1.0]
140,0.002400,0.010334,1.000000,1.000000,1.000000,[1.0],[1.0]
160,0.013800,0.013473,1.000000,1.000000,1.000000,[1.0],[1.0]
180,0.000100,0.011748,1.000000,1.000000,1.000000,[1.0],[1.0]
200,0.003600,0.009021,1.000000,1.000000,1.000000,[1.0],[1.0]


Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
Trainer is attempting to log a value of "[1.0]" of type <class 'list'> for key "eval/per_category_iou" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[1.0]" of type <class 'list'> for key "eval/per_category_accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
Trainer is attempting to log a value of "[1.0]" of type <class 'list'> for key "eval/per_category_iou" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[1.0]" of type <class 'list'> for key "eval/per_category_accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Do

Step,Training Loss,Validation Loss,Mean Iou,Mean Accuracy,Overall Accuracy,Per Category Iou,Per Category Accuracy
20,0.208200,0.068070,1.000000,1.000000,1.000000,[1.0],[1.0]
40,0.013000,0.031754,1.000000,1.000000,1.000000,[1.0],[1.0]
60,0.029000,0.021910,1.000000,1.000000,1.000000,[1.0],[1.0]
80,0.042000,0.012730,1.000000,1.000000,1.000000,[1.0],[1.0]
100,0.002200,0.016982,1.000000,1.000000,1.000000,[1.0],[1.0]
120,0.003900,0.022752,1.000000,1.000000,1.000000,[1.0],[1.0]
140,0.002400,0.010334,1.000000,1.000000,1.000000,[1.0],[1.0]
160,0.013800,0.013473,1.000000,1.000000,1.000000,[1.0],[1.0]
180,0.000100,0.011748,1.000000,1.000000,1.000000,[1.0],[1.0]
200,0.003600,0.009021,1.000000,1.000000,1.000000,[1.0],[1.0]


Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
Trainer is attempting to log a value of "[1.0]" of type <class 'list'> for key "eval/per_category_iou" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[1.0]" of type <class 'list'> for key "eval/per_category_accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
Trainer is attempting to log a value of "[1.0]" of type <class 'list'> for key "eval/per_category_iou" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[1.0]" of type <class 'list'> for key "eval/per_category_accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Do

In [1]:
'lol'

'lol'